In [ ]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from PIL import Image
from numpy import asarray

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import urllib

import time

In [ ]:
final1_df = pd.DataFrame(final1)
final1_df.to_excel('images.xlsx')

In [ ]:
images = pd.read_excel('images.xlsx')
images.columns = ['No.', 'Link']
images

In [ ]:
link_list = images["Link"].to_list()

Another way to scrape images is using bot_studio but it is not working very well?

In [ ]:
# Import library
from clustimage import Clustimage

# Initialize with default settings.
cl = Clustimage(method='pca', grayscale=False)

path = 'xx'
# load images
pathnames = path

# Run the model to find the optimal clusters.
results = cl.fit_transform(pathnames)

# Make scatter plots
cl.scatter()

# More input arguments for the scatterplot
cl.scatter(dotsize=35)


In [ ]:
results['img']

In [ ]:
# Import library
from clustimage import Clustimage

# init
cl = Clustimage(method='pca', grayscale=False)
# cl = Clustimage(method='pca-hog')

# load images
pathnames = path

# Cluster images using the input pathnames.
results = cl.fit_transform(pathnames, min_clust=3, max_clust=13)


In [ ]:
results.keys()

#### Detect unique images

In [ ]:
# Get the unique images
unique_samples = cl.unique()
#
print(unique_samples.keys())
# ['labels', 'idx', 'xycoord_center', 'pathnames']
#
# Collect the unique images from the input
pathnames[unique_samples['idx'],:]

# Plot unique images.
#cl.plot_unique()

In [ ]:
unique_samples['idx']

In [ ]:
# Plot all images per cluster
cl.plot(cmap='binary')


In [ ]:
# Make various plots:

# Silhouette plots
cl.clusteval.plot()
cl.clusteval.scatter(cl.results['xycoord'])

# PCA explained variance plot
# cl.pca.plot()

# Dendrogram
cl.dendrogram()

# Plot unique image per cluster
cl.plot_unique(img_mean=False)

# Scatterplot
cl.scatter(dotsize=50, zoom=0.5, img_mean=False)

# Plot images per cluster or all clusters
cl.plot()

In [ ]:
# Make scatterplot
cl.scatter(zoom=None)

# Plot the image that is in the center of the cluster
cl.scatter(zoom=4)

# Lets change some more arguments to make a pretty scatterplot
cl.scatter(zoom=None, dotsize=200, figsize=(25, 15), args_scatter={'fontsize':24, 'gradient':'#FFFFFF', 'cmap':'Set2', 'legend':True})

In [ ]:
# Import library
from clustimage import Clustimage

# init
cl = Clustimage(method='pca', grayscale=False, dim=(64,64))

# Load example with faces
X = cl.import_example(data='faces')

# Preproceesing, cluster detection
results = cl.fit_transform(X, min_clust=4, max_clust=20)

# Make various plots:

# Silhouette plots
cl.clusteval.plot()
cl.clusteval.scatter(cl.results['xycoord'])

# Dendrogram
cl.dendrogram()

# Plot unique image per cluster
cl.plot_unique(img_mean=False)

# Scatterplot
cl.scatter(dotsize=50, zoom=0.5, img_mean=False)

# Plot images per cluster or all clusters
cl.plot(labels=15)

### Cluster using Vision Transformers

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image as img
from IPython.display import Image, display

In [ ]:

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
     

In [ ]:
image_array = img.open('pictures/image4.jpg')

In [ ]:
inputs = feature_extractor(images = image_array, 
                           return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits

In [ ]:
predicted_class_idx = logits.argmax(-1).item()

print("Predicted class:", model.config.id2label[predicted_class_idx])

### Cluster using VVG16

In [ ]:
!pip install --upgrade tensorflow --user


In [ ]:
# for loading/processing the images  
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

In [ ]:
# state path
path = 'pictures/'

# this list holds all the image filename
images = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpg'):
          # adds only the image files to the flowers list
            images.append('pictures/' + file.name)

In [ ]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

In [ ]:
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [ ]:
data = {}
p = r"pictures/features"

# lop through each image in the dataset
for img in images:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(img,model)
        data[img] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)

In [ ]:
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))
feat.shape

In [ ]:
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))

# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)

In [ ]:
feat.shape

In [ ]:
# reduce the amount of dimensions in the feature vector
pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

In [ ]:
# cluster feature vectors
kmeans = KMeans(n_clusters=10, random_state=22)
kmeans.fit(x)

In [ ]:
# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)


In [ ]:
# function that lets you view a cluster (based on identifier)        
def view_cluster(cluster):
    plt.figure(figsize = (80,80));
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 100:
        print(f"Clipping cluster size from {len(files)} to 50")
        files = files[:99]
    # plot each image in the cluster
    
    for index, file in enumerate(files):
        #plt.title("Cluster " + str(cluster))
        plt.subplot(10,10,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')

In [ ]:
# this is just incase you want to see which value for k might be the best 
sse = []
list_k = list(range(3, 50))

for k in list_k:
    km = KMeans(n_clusters=k, random_state=22)
    km.fit(x)
    
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse)
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

In [ ]:
view_cluster(0)

In [ ]:
view_cluster(1)

In [ ]:
view_cluster(2)

In [ ]:
view_cluster(3)

In [ ]:
view_cluster(4)

In [ ]:
view_cluster(5)

In [ ]:
view_cluster(6)

In [ ]:
view_cluster(7)

In [ ]:
view_cluster(8)

In [ ]:
view_cluster(9)

### Cluster using tSNE

In [ ]:
feat.shape

In [ ]:
import random
from sklearn.manifold import TSNE

In [ ]:
num_images_to_plot = 1000

if len(images) > num_images_to_plot:
    sort_order = sorted(random.sample(range(len(images)), num_images_to_plot))
    images = [images[i] for i in sort_order]
    pca_features = [feat[i] for i in sort_order]

In [ ]:
X = np.array(feat)
tsne = TSNE(n_components=2, learning_rate=150, perplexity=30, angle=0.2, verbose=2).fit_transform(X)

In [ ]:
tx, ty = tsne[:,0], tsne[:,1]
tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
width = 4000
height = 3000
max_dim = 100

full_image = Image.new('RGBA', (width, height))
for img, x, y in zip(images, tx, ty):
    tile = Image.open(img)
    rs = max(1, tile.width/max_dim, tile.height/max_dim)
    tile = tile.resize((int(tile.width/rs), int(tile.height/rs)), Image.ANTIALIAS)
    full_image.paste(tile, (int((width-max_dim)*x), int((height-max_dim)*y)), mask=tile.convert('RGBA'))

plt.figure(figsize = (16,12))
plt.imshow(full_image)

In [ ]:
X = np.array(feat[:1000])
tsne = TSNE(n_components=2, learning_rate=150, perplexity=30, angle=0.2, verbose=2).fit_transform(X)

In [ ]:
import rasterfairy

# nx * ny = 1000, the number of images
nx = 40
ny = 25

# assign to grid
grid_assignment = rasterfairy.transformPointCloud2D(tsne, target=(nx, ny))

In [ ]:
tile_width = 72
tile_height = 56

full_width = tile_width * nx
full_height = tile_height * ny
aspect_ratio = float(tile_width) / tile_height

grid_image = Image.new('RGB', (full_width, full_height))

for img, grid_pos in zip(images, grid_assignment[0]):
    idx_x, idx_y = grid_pos
    x, y = tile_width * idx_x, tile_height * idx_y
    tile = Image.open(img)
    tile_ar = float(tile.width) / tile.height  # center-crop the tile to match aspect_ratio
    if (tile_ar > aspect_ratio):
        margin = 0.5 * (tile.width - aspect_ratio * tile.height)
        tile = tile.crop((margin, 0, margin + aspect_ratio * tile.height, tile.height))
    else:
        margin = 0.5 * (tile.height - float(tile.width) / aspect_ratio)
        tile = tile.crop((0, margin, tile.width, margin + float(tile.width) / aspect_ratio))
    tile = tile.resize((tile_width, tile_height), Image.ANTIALIAS)
    grid_image.paste(tile, (int(x), int(y)))

plt.figure(figsize = (16,12))
plt.imshow(grid_image)

### TensorBoard

In [ ]:
os.add_dll_directory(r"C:\Program Files (x86)\NVIDIA Corporation\PhysX\Engine\73BE2921551D\ImplicitLibs\cudart32_65.dll")

In [ ]:
import os
import re
import csv
from PIL import Image
import pandas as pd
import torch
import torchvision.models as models
import torchvision.transforms as transforms

def get_vector(input_image):
    image = input_image.convert("RGB")  # in case input image is not in RGB format
    img_t = transform(image)
    batch_t = torch.unsqueeze(img_t, 0)
    my_embedding = torch.zeros([1, 512, 1, 1])
    return my_embedding
    
def copy_data(m, i, o):
    my_embedding.copy_(o.data)
    h = layer.register_forward_hook(copy_data)
    model(batch_t)
    h.remove()
    return my_embedding.squeeze().cpu().numpy()

model = models.resnet18(pretrained=True)
layer = model._modules.get('avgpool')
model.eval()

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
im_path = 'pictures/'
im_names = [os.path.join(root, name)
            for root, dirs, files in os.walk(im_path)
            for name in files
            if name.endswith(".jpg")]
existing_images_df = pd.DataFrame([re.findall(r"[\w']+", im_name)[1:3] for im_name in im_names],
                                  columns=['cat_id', 'pid'])
existing_images_df['impath'] = im_names
vecs = [list(get_vector(Image.open(impath))) for _, pid, impath in existing_images_df.values]
with open('vis/feature_vecs.tsv', 'w') as fw:
    csv_writer = csv.writer(fw, delimiter='\t')
    csv_writer.writerows(vecs)

In [ ]:
images = [Image.open(filename).resize((300,300)) for filename in existing_images_df['impath']]
image_width, image_height = images[0].size
one_square_size = int(np.ceil(np.sqrt(len(images))))
master_width = (image_width * one_square_size) 
master_height = image_height * one_square_size
spriteimage = Image.new(
    mode='RGBA',
    size=(master_width, master_height),
    color=(0,0,0,0))  # fully transparent
for count, image in enumerate(images):
    div, mod = divmod(count,one_square_size)
    h_loc = image_width*div
    w_loc = image_width*mod    
    spriteimage.paste(image,(w_loc,h_loc))
spriteimage.convert("RGB").save('vis/sprite.jpg', transparency=0)

In [ ]:
metadata = existing_images_df[['cat_id', 'pid']].to_csv('vis/metadata.tsv', sep='\t', index=False)

In [ ]:
import tensorflow
#import tensorflow-gpu

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
!pip install --ignore-installed tf-nightly --user

In [ ]:
%load_ext tensorboard

In [ ]:
file_writer = tf.summary.create_file_writer('file path', sess.graph)

In [ ]:
%tensorboard --logdir ./vis

In [ ]:
%tensorboard --logdir ./vis --host localhost --port 8098

In [ ]:
from tensorboard import notebook
notebook.list()

In [ ]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000)

In [ ]:
%load_ext tensorboard
import csv
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import re
import csv
from PIL import Image
import pandas as pd
import torch
import torchvision.models as models
import torchvision.transforms as transforms

In [ ]:
model = models.resnet18(pretrained=True)
layer = model._modules.get('avgpool')
model.eval()

In [ ]:
embeddings = tf.keras.models.Model( 
    inputs=model.inputs, 
    outputs=model.layers[-1].output 
)